In [8]:
import pandas as pd
import glob
import os
import re # For regular expressions to extract ticker from filename

def load_csv_files_from_folder(folder_path):
    """
    Loads all CSV files from a given folder into a list of DataFrames.
    Adds a 'Ticker' column to each DataFrame, inferred from the filename.
    Ensures 'Date' column is datetime index.
    """
    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    loaded_dataframes = []

    if not csv_files:
        print(f"No CSV files found in '{folder_path}'.")
        return loaded_dataframes

    print(f"Attempting to load {len(csv_files)} CSV files from '{folder_path}'...")
    for file_path in csv_files:
        try:
            df = pd.read_csv(file_path)

            # --- Extract Ticker from filename and add as a column ---
            # Corrected Regex: Matches characters at the start until an underscore, then '.csv'
            # For 'MSFT_historical_data.csv', it will capture 'MSFT'.
            filename = os.path.basename(file_path)
            ticker_match = re.match(r'([A-Za-z0-9]+)_historical_data\.csv', filename)
            if ticker_match:
                ticker_symbol = ticker_match.group(1).upper()
                df['Ticker'] = ticker_symbol
            else:
                print(f"Warning: Could not extract ticker from filename '{filename}'. Skipping this file.")
                continue # Skip if ticker can't be identified

            # --- Ensure 'Date' column is datetime and set as index ---
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
                df.dropna(subset=['Date'], inplace=True) # Remove rows where Date couldn't be parsed
                df.set_index('Date', inplace=True)
                df.sort_index(inplace=True) # Ensure chronological order
            else:
                print(f"Warning: 'Date' column not found in '{filename}'. Skipping this file.")
                continue

            loaded_dataframes.append(df)
            print(f"  Loaded {ticker_symbol} with {len(df)} rows.")

        except Exception as e:
            print(f"Error loading or processing '{file_path}': {e}. Skipping.")
            pass

    return loaded_dataframes

# --- Execution for Stock Data ---
# PLEASE ENSURE THIS PATH IS CORRECT FOR YOUR STOCK CSVs
yfinance_data_folder = '../data/yfinance_data/' 
all_yfinance_dfs = load_csv_files_from_folder(yfinance_data_folder)

print(f"\nSuccessfully loaded {len(all_yfinance_dfs)} Stock DataFrames.")
if all_yfinance_dfs:
    print("\nHead of the first loaded Stock DataFrame (should now have 'Ticker' column and DatetimeIndex):")
    print(all_yfinance_dfs[0].head())
    print(all_yfinance_dfs[0].info())


# --- Execution for News Data ---
# PLEASE ENSURE THIS PATH IS CORRECT FOR YOUR NEWS CSV
news_file_path = '../data/raw_analyst_ratings.csv/raw_analyst_ratings.csv'
try:
    df_news_original = pd.read_csv(news_file_path) # Renamed to df_news_original for clarity
    print(f"\n\nDataset loaded successfully from '{news_file_path}'. Displaying the first 5 rows and info:")
    print(df_news_original.head(5))
    print("\n")
    df_news_original.info()
except FileNotFoundError:
    print(f"Error: '{news_file_path}' not found. Please check the path.")
    exit()
except Exception as e:
    print(f"An unexpected error occurred while loading the news dataset: {e}")
    exit()

# From now on, we will work with 'df_news_original' for news processing

Attempting to load 7 CSV files from '../data/yfinance_data/'...
  Loaded MSFT with 9672 rows.
  Loaded NVDA with 6421 rows.
  Loaded AMZN with 6846 rows.
  Loaded TSLA with 3545 rows.
  Loaded META with 2926 rows.
  Loaded AAPL with 10998 rows.
  Loaded GOOG with 5020 rows.

Successfully loaded 7 Stock DataFrames.

Head of the first loaded Stock DataFrame (should now have 'Ticker' column and DatetimeIndex):
                Open      High       Low     Close  Adj Close      Volume  \
Date                                                                        
1986-03-13  0.088542  0.101563  0.088542  0.097222   0.059946  1031788800   
1986-03-14  0.097222  0.102431  0.097222  0.100694   0.062087   308160000   
1986-03-17  0.100694  0.103299  0.100694  0.102431   0.063158   133171200   
1986-03-18  0.102431  0.103299  0.098958  0.099826   0.061552    67766400   
1986-03-19  0.099826  0.100694  0.097222  0.098090   0.060482    47894400   

            Dividends  Stock Splits Ticker  
Date

In [13]:
import pandas as pd
# Assuming df_news_original is loaded from the previous block

print("\n--- Starting News Data Normalization and Alignment for 'df_news_original' ---")

# --- Step 1: Standardize 'date' Column to Datetime Objects ---
# Explanation: Your news 'date' column contains full timestamps with timezone offsets
# (e.g., '2020-06-05 10:30:54-04:00'). For Pandas to efficiently work with dates
# and for proper alignment, it must be converted into a standardized `datetime` format.
# `errors='coerce'` is crucial: if any date string cannot be parsed, it will be
# converted to `NaT` (Not a Time/Date) instead of raising an error.
# We then drop these `NaT` entries because they cannot be aligned.
# Note: pd.to_datetime handles timezone offsets automatically, resulting in timezone-aware datetimes.
print("\n--- Step 1: Converting 'date' column to Datetime ---")
df_news_processed = df_news_original.copy() # Work on a copy of the original news DataFrame
df_news_processed['date'] = pd.to_datetime(df_news_processed['date'], errors='coerce')
# Drop rows where date parsing failed (i.e., 'date' became NaT)
df_news_processed.dropna(subset=['date'], inplace=True)

print("News DataFrame after converting 'date' to datetime and dropping NaTs:")
print(df_news_processed.head())
print(df_news_processed.info())


# --- Step 2: Normalize Timestamps to Date Only ---
# Explanation: Stock data is typically daily (one data point per day) and does not
# include time components or timezone information (it's usually market-specific).
# Your news dates currently include exact times and timezone offsets.
# To align news perfectly with daily stock data, we need to strip both the time
# component and the timezone information, leaving only the calendar date
# (e.g., '2020-06-05 10:30:54-04:00' becomes '2020-06-05 00:00:00').
# `.dt.normalize()` handles this: it converts to timezone-naive datetime and sets time to midnight.
print("\n--- Step 2: Normalizing Timestamps to Date Only ---")
df_news_processed['aligned_date'] = df_news_processed['date'].dt.normalize()
print("News DataFrame with 'aligned_date' (date-only, timezone-naive):")
print(df_news_processed.head())
print(df_news_processed.info())


# --- Step 3: Standardize 'stock' Column ---
# Explanation: This column identifies which company the news refers to. It's crucial
# that these entries (e.g., 'A', 'MSFT') are clean and consistent. Stock ticker
# symbols are typically uppercase. We'll convert them to string type, then to
# uppercase, and strip any leading/trailing whitespace to ensure uniformity.
# We'll also drop any rows where the 'stock' symbol is missing or becomes empty,
# as we cannot link such news to a specific company.
print("\n--- Step 3: Cleaning and Standardizing 'stock' column ---")
df_news_processed['stock'] = df_news_processed['stock'].astype(str).str.upper().str.strip()
df_news_processed.dropna(subset=['stock'], inplace=True) # Drop if 'stock' is NaN/empty after cleaning

# It's a good idea to ensure only the desired big tech tickers are present if you don't
# want to process news for other stocks like 'A' (Agilent)
desired_big_tech_tickers = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA']
initial_rows = len(df_news_processed)
df_news_processed = df_news_processed[df_news_processed['stock'].isin(desired_big_tech_tickers)].copy()
print(f"Filtered for desired big tech tickers. Removed {initial_rows - len(df_news_processed)} rows.")
print("News DataFrame after standardizing and filtering 'stock' column:")
print(df_news_processed.head())
print(df_news_processed.info())


# --- Step 4: Aggregate News per Day per Ticker ---
# Explanation: On any given 'aligned_date', there might be multiple news articles
# for the same stock. To align with a single daily stock price entry,
# you need to summarize these multiple news items into a single daily record.
# This aggregation consolidates all news for a specific stock on a particular day.
# Here, we'll count the number of articles and take the first headline published
# on that day for that stock. You could also include sentiment analysis here
# if you have a numerical sentiment score derived from your 'sentiment' column.
print("\n--- Step 4: Aggregating News per Day per Ticker ---")
aggregated_news_df = df_news_processed.groupby(['aligned_date', 'stock']).agg(
    news_count=('headline', 'size'), # Count the number of news articles
    first_headline=('headline', lambda x: x.iloc[0]) # Get the first headline of the day for that stock
    # If you have a 'sentiment' column and have converted it to numerical scores (e.g., -1, 0, 1),
    # you could add: 'avg_sentiment': ('sentiment_score', 'mean')
).reset_index() # reset_index moves 'aligned_date' and 'stock' back into regular columns

print("Aggregated News DataFrame (first 5 rows):")
print(aggregated_news_df.head())
print(aggregated_news_df.info())


# --- Step 5: Final Preparation: Rename 'aligned_date' to 'Date' and Set as Index ---
# Explanation: For efficient and clean merging with your stock DataFrames
# (which have their 'Date' as the index, as modified in Step 0), it's a best practice
# to also rename the aggregated date column to 'Date' and set it as the index
# of this news DataFrame. This prepares it for a direct index-to-index merge.
print("\n--- Step 5: Final Preparation: Rename 'aligned_date' to 'Date' and Set as Index ---")
aggregated_news_df.rename(columns={'aligned_date': 'Date'}, inplace=True)
aggregated_news_df.set_index('Date', inplace=True)

print("Final Aggregated News DataFrame ready for merging (first 5 rows):")
print(aggregated_news_df.head())
print(aggregated_news_df.info())

print("\n--- News Data Normalization and Alignment Complete ---")
print("The 'aggregated_news_df' is now prepared and ready to be merged with your historical stock data (all_yfinance_dfs).")


--- Starting News Data Normalization and Alignment for 'df_news_original' ---

--- Step 1: Converting 'date' column to Datetime ---
News DataFrame after converting 'date' to datetime and dropping NaTs:
   Unnamed: 0                                           headline  \
0           0            Stocks That Hit 52-Week Highs On Friday   
1           1         Stocks That Hit 52-Week Highs On Wednesday   
2           2                      71 Biggest Movers From Friday   
3           3       46 Stocks Moving In Friday's Mid-Day Session   
4           4  B of A Securities Maintains Neutral on Agilent...   

                                                 url          publisher  \
0  https://www.benzinga.com/news/20/06/16190091/s...  Benzinga Insights   
1  https://www.benzinga.com/news/20/06/16170189/s...  Benzinga Insights   
2  https://www.benzinga.com/news/20/05/16103463/7...         Lisa Levin   
3  https://www.benzinga.com/news/20/05/16095921/4...         Lisa Levin   
4  https://ww

In [11]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /home/ca/nltk_data...


True

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# --- Ensure VADER lexicon is downloaded ---
try:
    nltk.data.find('sentiment/vader_lexicon.zip')
except nltk.downloader.DownloadError:
    print("VADER lexicon not found. Downloading...")
    nltk.download('vader_lexicon')
    print("VADER lexicon downloaded.")

print("\n--- Starting Sentiment Analysis on News Headlines ---")

# --- Initialize the VADER sentiment analyzer ---
analyzer = SentimentIntensityAnalyzer()

# --- Re-create df_news_processed to ensure it's in the correct state ---

# Load the original news data again (assuming it's available from previous blocks)
news_file_path = '../data/raw_analyst_ratings.csv/raw_analyst_ratings.csv'
try:
    df_news_original = pd.read_csv(news_file_path)
except FileNotFoundError:
    print(f"Error: '{news_file_path}' not found. Cannot perform sentiment analysis.")
    exit()
except Exception as e:
    print(f"An error occurred loading news data for sentiment analysis: {e}")
    exit()

df_news_processed = df_news_original.copy()

# Step 1: Standardize 'date' column to Datetime Objects
df_news_processed['date'] = pd.to_datetime(df_news_processed['date'], errors='coerce')
df_news_processed.dropna(subset=['date'], inplace=True)

# Step 2: Normalize Timestamps to Date Only
df_news_processed['aligned_date'] = df_news_processed['date'].dt.normalize()

# Step 3: Standardize 'stock' Column
df_news_processed['stock'] = df_news_processed['stock'].astype(str).str.upper().str.strip()
df_news_processed.dropna(subset=['stock'], inplace=True)
desired_big_tech_tickers = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA']
df_news_processed = df_news_processed[df_news_processed['stock'].isin(desired_big_tech_tickers)].copy()
# Ensure 'headline' column is string type and handle potential NaNs before sentiment analysis
df_news_processed['headline'] = df_news_processed['headline'].astype(str).fillna('')

print("\nDataFrame state before sentiment analysis (first 5 rows with relevant columns):")
print(df_news_processed[['date', 'aligned_date', 'stock', 'headline']].head())
print(df_news_processed.info())

# --- Step 1: Calculate Sentiment Scores for Each Headline ---
# Explanation: We define a function that takes a text (headline) and uses VADER
# to return a 'compound' sentiment score. The compound score is a normalized
# weighted composite score, typically ranging from -1 (most negative) to +1 (most positive).
# A score closer to 0 indicates neutrality.
print("\n--- Step 1: Calculating VADER Compound Sentiment Scores ---")

def get_vader_sentiment_score(text):
    if not isinstance(text, str) or not text:
        return 0.0 # Return 0 for empty or non-string headlines
    return analyzer.polarity_scores(text)['compound']

# Apply the function to the 'headline' column to create a new 'sentiment_score' column
df_news_processed['sentiment_score'] = df_news_processed['headline'].apply(get_vader_sentiment_score)

print("News DataFrame with 'sentiment_score' (first 5 rows):")
print(df_news_processed[['headline', 'sentiment_score']].head())


# --- Step 2: Categorize Sentiment (Optional but helpful for interpretation) ---
# Explanation: While a numerical score is good for quantitative analysis,
# it's often useful to categorize sentiment into labels like 'positive', 'negative',
# and 'neutral' for easier understanding and visualization.
# We'll use common thresholds:
#   - score >= 0.05: Positive
#   - score <= -0.05: Negative
#   - otherwise: Neutral (scores between -0.05 and 0.05)
print("\n--- Step 2: Categorizing Sentiment Labels ---")

def get_sentiment_label(score):
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the categorization function to create a new 'sentiment_label' column
df_news_processed['sentiment_label'] = df_news_processed['sentiment_score'].apply(get_sentiment_label)

print("News DataFrame with 'sentiment_label' (first 5 rows):")
print(df_news_processed[['headline', 'sentiment_score', 'sentiment_label']].head())

print("\nSummary of sentiment labels:")
print(df_news_processed['sentiment_label'].value_counts())

print("\nSentiment analysis complete for individual news headlines.")
print("The 'df_news_processed' DataFrame now contains 'sentiment_score' and 'sentiment_label' columns.")



--- Starting Sentiment Analysis on News Headlines ---

DataFrame state before sentiment analysis (first 5 rows with relevant columns):
                          date              aligned_date stock  \
6680 2020-06-10 11:33:26-04:00 2020-06-10 00:00:00-04:00  AAPL   
6681 2020-06-10 08:14:08-04:00 2020-06-10 00:00:00-04:00  AAPL   
6682 2020-06-10 07:53:47-04:00 2020-06-10 00:00:00-04:00  AAPL   
6683 2020-06-10 07:19:25-04:00 2020-06-10 00:00:00-04:00  AAPL   
6684 2020-06-10 06:27:11-04:00 2020-06-10 00:00:00-04:00  AAPL   

                                               headline  
6680  Tech Stocks And FAANGS Strong Again To Start D...  
6681      10 Biggest Price Target Changes For Wednesday  
6682  Benzinga Pro's Top 5 Stocks To Watch For Wed.,...  
6683  Deutsche Bank Maintains Buy on Apple, Raises P...  
6684  Apple To Let Users Trade In Their Mac Computer...  
<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 6680 to 1255230
Data columns (total 7 columns):
 #   Column   

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re # For regular expressions (used in stock data loading if needed)
import os # For file path operations
import glob # For listing files

# --- 0. Ensure NLTK VADER lexicon is downloaded ---
try:
    nltk.data.find('sentiment/vader_lexicon.zip')
except nltk.downloader.DownloadError:
    print("VADER lexicon not found. Downloading...")
    nltk.download('vader_lexicon')
    print("VADER lexicon downloaded.")

print("--- Starting Full News Data Processing with Sentiment Analysis ---")

# --- 1. Load the News Data (as per your specified path) ---
news_file_path = '../data/raw_analyst_ratings.csv/raw_analyst_ratings.csv'
try:
    df_news_original = pd.read_csv(news_file_path)
    print(f"\nNews dataset loaded successfully from '{news_file_path}'.")
except FileNotFoundError:
    print(f"Error: News file '{news_file_path}' not found. Please check the path.")
    exit()
except Exception as e:
    print(f"An error occurred loading news data: {e}")
    exit()

print("Original News DataFrame head:")
print(df_news_original.head())
print("\nOriginal News DataFrame info:")
df_news_original.info()


# --- 2. Initial News Data Processing (Steps from previous alignment) ---
# Create a working copy
df_news_processed = df_news_original.copy()

# Step 2.1: Standardize 'date' column to Datetime Objects
df_news_processed['date'] = pd.to_datetime(df_news_processed['date'], errors='coerce')
df_news_processed.dropna(subset=['date'], inplace=True)
print("\nAfter date conversion and NaT drop:")
print(df_news_processed.info())

# Step 2.2: Normalize Timestamps to Date Only
df_news_processed['aligned_date'] = df_news_processed['date'].dt.normalize()
print("\nAfter date normalization:")
print(df_news_processed.info())

# Step 2.3: Standardize 'stock' Column and filter for big tech
df_news_processed['stock'] = df_news_processed['stock'].astype(str).str.upper().str.strip()
df_news_processed.dropna(subset=['stock'], inplace=True)
desired_big_tech_tickers = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA'] # Ensure these match your stock files
initial_rows = len(df_news_processed)
df_news_processed = df_news_processed[df_news_processed['stock'].isin(desired_big_tech_tickers)].copy()
print(f"\nAfter stock standardization and filtering for {len(desired_big_tech_tickers)} big tech tickers. Removed {initial_rows - len(df_news_processed)} rows.")
print(df_news_processed.info())

# Ensure 'headline' column is string type and handle potential NaNs before sentiment analysis
df_news_processed['headline'] = df_news_processed['headline'].astype(str).fillna('')


# --- 3. Sentiment Analysis on Headlines ---
print("\n--- Performing Sentiment Analysis on Headlines ---")
analyzer = SentimentIntensityAnalyzer()

def get_vader_sentiment_score(text):
    if not isinstance(text, str) or not text:
        return 0.0 # Return 0 for empty or non-string headlines
    return analyzer.polarity_scores(text)['compound']

df_news_processed['sentiment_score'] = df_news_processed['headline'].apply(get_vader_sentiment_score)

print("News DataFrame with 'sentiment_score' (first 5 rows with headline and score):")
print(df_news_processed[['headline', 'sentiment_score']].head())
print(f"Sentiment analysis complete. Added 'sentiment_score' column. Total rows: {len(df_news_processed)}")


# --- 4. Re-aggregate News per Day per Ticker (INCLUDING SENTIMENT) ---
print("\n--- Re-aggregating News Data, including Mean Sentiment Score ---")
aggregated_news_df = df_news_processed.groupby(['aligned_date', 'stock']).agg(
    news_count=('headline', 'size'), # Count number of headlines
    first_headline=('headline', lambda x: x.iloc[0]), # Get the first headline
    avg_sentiment=('sentiment_score', 'mean') # Calculate the mean sentiment for the day/stock
).reset_index() # Moves 'aligned_date' and 'stock' back to columns

print("Aggregated News DataFrame (first 5 rows, including avg_sentiment):")
print(aggregated_news_df.head())
print(f"Aggregated DataFrame shape: {aggregated_news_df.shape}")


# --- 5. Final Preparation: Rename 'aligned_date' to 'Date' and Set as Index ---
print("\n--- Final Preparation: Renaming 'aligned_date' to 'Date' and Setting as Index ---")
aggregated_news_df.rename(columns={'aligned_date': 'Date'}, inplace=True)
aggregated_news_df.set_index('Date', inplace=True)

print("Final 'aggregated_news_df' ready for merging with stock data:")
print(aggregated_news_df.head(10)) # Display more rows to see some variety
print("\nFinal 'aggregated_news_df' info:")
print(aggregated_news_df.info())

print("\n--- News Data Processing with Sentiment Analysis Complete ---")
print("You can now see the 'news_count' and 'avg_sentiment' for each stock on each trading day.")

--- Starting Full News Data Processing with Sentiment Analysis ---
